During this notebook you can see how to configure and use a Amazon Cognito's user pool to create a user creation flow with the following requirements:
* Email and phone number are required in user creation. They must be unique in the user pool.
* The email is the login username.
* A code will be generated to verify the user's email address and the phone number. They must be verified before activate the user.

Let's import the boto3 library and configure a client to connect to cognito. This requires `aws-cli` exists in your system. In this example we're using the default profile configured in my system, but you can use a different profile by using the following:
```
session = boto3.session.Session(profile_name='your_profile')
cognito = session.client('cognito-idp')
```

In [43]:
import boto3
cognito = boto3.client('cognito-idp')

This is a function to prettify the user print output


In [44]:

def prettify_user_output(user):
    print_user = {}
    print_user['username'] = user['Username']
    for attr in user['UserAttributes']:
        if attr['Name'] == 'email':
            print_user['email'] = attr['Value']
        elif attr['Name'] == 'phone_number':
            print_user['phone_number'] = attr['Value']
    print_user['enabled'] = user['Enabled']
    print_user['user_status'] = user['UserStatus']

    print(f"""
    {'-'*40}
    Username: {print_user['username']}
    Email: {print_user['email']}
    Phone Number: {print_user['phone_number']}
    User Status: {print_user['user_status']}
    Enable: {print_user['enabled']}
    {'-'*40}
    """
    )

At this point we will create the user pool if not created yet. It created, comment the next block and user your own `user_pool_id` for the following requests.

In [46]:
user_pool_response = cognito.create_user_pool(
    PoolName='TestUserPool',
    AutoVerifiedAttributes=['email'],
    UsernameAttributes=['phone_number','email'],
    MfaConfiguration='OFF',
    UserAttributeUpdateSettings={
        'AttributesRequireVerificationBeforeUpdate': ['email']
    },
    EmailConfiguration={
        'EmailSendingAccount': 'COGNITO_DEFAULT'
    },
    AdminCreateUserConfig={
        'AllowAdminCreateUserOnly': False
    },
    Schema=[
        {
            'Name': 'department',
            'AttributeDataType': 'String',
            'DeveloperOnlyAttribute': False,
            'Mutable': True,
            'Required': False,
            'StringAttributeConstraints': {
                'MinLength': '0',
                'MaxLength': '100'
            }
        }
    ],
    UserPoolAddOns={
        'AdvancedSecurityMode': 'OFF',
    },
    UsernameConfiguration={
        'CaseSensitive': False
    }
)

In [47]:
user_pool_id = user_pool_response['UserPool']['Id']
print("User pool ID: " + user_pool_id)

User pool ID: us-east-1_IecKXZb05


In order to use an external application it's necessary to create a new client for our user pool. If you already have your own, just comment the next block and add your `client_id`.

In [48]:
client_response = cognito.create_user_pool_client(
    UserPoolId=user_pool_id,
    ClientName='TestClient',
    CallbackURLs=[
        'http://localhost',
    ]
)

In [49]:
client_id = client_response['UserPoolClient']['ClientId']
print("Client Id: " + client_id)

Client Id: 4uee0udf1d187ut3vb2ktafg20


Now, let's create our first user by using the email address as username.

In [50]:
user_pass = '@ComplexPass1'
user_mail = 'your_valid@email.com'
user_phone_number = '+1234234234'

In [51]:
sing_up_response = cognito.sign_up(
    ClientId=client_id,
    Username=user_mail,
    Password=user_pass,
    UserAttributes=[
        {
            'Name': 'email',
            'Value': user_mail
        },
        {
            'Name': 'phone_number',
            'Value': user_phone_number
        },
        {
            'Name': 'custom:department',
            'Value': 'example_department'
        }
    ]
)

Let's get the user to double-check that everything is working well

In [55]:
user = cognito.admin_get_user(
    UserPoolId=user_pool_id,
    Username=user_mail
)

prettify_user_output(user)



    ----------------------------------------
    Username: 8b79d87d-3883-47b0-9b18-f2791d07c4e5
    Email: your_valid@email.com
    Phone Number: +1234234234
    User Status: UNCONFIRMED
    Enable: True
    ----------------------------------------
    


A new email was sent to our inbox containing the sing-up code. Use it to fill out the `code` parameter and confirm the sing-up operation.

In [ ]:
code = '######'
confirmation_response = cognito.confirm_sign_up(
    ClientId=client_id,
    Username=user_mail,
    ConfirmationCode=code,
    ForceAliasCreation=False
)

In [ ]:
user = cognito.admin_get_user(
    UserPoolId=user_pool_id,
    Username=user_mail
)

prettify_user_output(user)